In [34]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as rSquared
from keras.models import Sequential
from keras.layers import Conv2D, Dense



from tensorflow.keras.applications import VGG16

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [35]:
path='new_CNN_Datacube.pkl'

In [36]:
with open(path,'rb') as f:
    data = pickle.load(f)

In [37]:
(data[10][0])

{'Image': array([[[ 87,  94,  99, 255],
         [ 76,  81,  97, 255],
         [ 72,  79,  95, 255],
         ...,
         [ 87, 101,  98, 255],
         [ 81,  90,  93, 255],
         [ 80,  84,  91, 255]],
 
        [[111, 105, 101, 255],
         [106,  94,  96, 255],
         [ 91,  77,  91, 255],
         ...,
         [ 86, 110, 102, 255],
         [ 84, 103,  97, 255],
         [ 80,  78,  91, 255]],
 
        [[118, 106, 102, 255],
         [113,  92,  98, 255],
         [106,  85,  94, 255],
         ...,
         [ 84, 105, 105, 255],
         [ 92, 101, 105, 255],
         [ 96,  88,  98, 255]],
 
        ...,
 
        [[ 60,  81,  90, 255],
         [ 68,  84,  99, 255],
         [ 71,  82, 106, 255],
         ...,
         [135, 119, 117, 255],
         [146, 122, 118, 255],
         [120, 107, 107, 255]],
 
        [[ 63,  79,  98, 255],
         [ 66,  83,  99, 255],
         [ 70,  80,  97, 255],
         ...,
         [128, 110, 114, 255],
         [148, 110, 115, 2

In [38]:
NearRoad_lst = []
count = 0
for i in data:
    for j in i:
        count = count +1
        NearRoad_lst.append(j['Near_road'])

len(NearRoad_lst)

1422

In [39]:
UrbanIndex_lst = []
count = 0
for i in data:
    for j in i:
        count = count +1
        UrbanIndex_lst.append(j['Urban_index'])

len(UrbanIndex_lst)

1422

In [40]:
GreenIndex_lst = []
count = 0
for i in data:
    for j in i:
        count = count +1
        GreenIndex_lst.append(j['Green_index'])

len(GreenIndex_lst)

1422

In [41]:
PM_lst = []
count = 0
for i in data:
    for j in i:
        count = count +1
        PM_lst.append(j['PM'])

len(PM_lst)


1422

In [42]:
Image_lst = []
image_size = 224
count = 0
for i in data:
    for j in i:
        count = count +1
        Image_lst.append(j['Image'][:,:,:3])
        #print(count," ",j['PM'])
        #print(Imagess)

Imagearr = np.array(Image_lst)
Scaled_imageset=Imagearr*(1./255)


MemoryError: Unable to allocate 1.59 GiB for an array with shape (1422, 224, 224, 3) and data type float64

In [ ]:
## Train-Test Split
dataset = list(zip(Scaled_imageset,PM_lst))

###shuffle dataset to get train, test datasets
random.Random(42).shuffle(dataset)
#random.shuffle(dataset)
batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = int(np.ceil(0.8 * total_batches))
test_batches = total_batches - train_batches
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]



In [ ]:
(test[4][0]*(1.*255)).astype('uint8')

In [ ]:
(test[5][0]*(1.*255)).astype('uint8')

In [ ]:
X_train_img = np.array([item[0] for item in train]).reshape(-1,image_size,image_size,3)
X_test_img = np.array([item[0] for item in test]).reshape(-1,image_size,image_size,3)

y_train = np.array([item[1] for item in train]).reshape(-1)
y_test = np.array([item[1] for item in test]).reshape(-1)

'''X_train_auxiliary1 = np.array([item[2] for item in train]).reshape((len(train)),1)
X_test_auxiliary1= np.array([item[2] for item in test]).reshape((len(test)),1)

X_train_auxiliary2 = np.array([item[3] for item in train]).reshape((len(train)),1)
X_test_auxiliary2= np.array([item[3] for item in test]).reshape((len(test)),1)

X_train_auxiliary3 = np.array([item[4] for item in train]).reshape((len(train)),1)
X_test_auxiliary3= np.array([item[4] for item in test]).reshape((len(test)),1)'''

In [ ]:
np.array(X_train_img).shape


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model and add custom layers
model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1)  # Output layer for PM2.5 prediction
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mae', metrics=['mae'])
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [ ]:
epochsize = 10
#model = model.fit(X_train, y_train, validation_split= 0.1,epochs=10, batch_size = 100, callbacks=[early_stop],verbose= 1,shuffle=False)
history = model.fit(x=X_train_img, y=y_train, validation_split= 0.1,epochs=epochsize, batch_size=100, callbacks=[early_stop],verbose= 1,shuffle=False)

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'],'b', label='training history')
pyplot.plot(history.history['val_loss'],'r',label='testing history')
pyplot.title("Train and Test Loss function")
pyplot.legend()
pyplot.show()

In [ ]:
loss = model.evaluate([X_test_img,X_test_auxiliary1,X_test_auxiliary2,X_test_auxiliary3],y_test)
#accuracy = model.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape

y_pred1 = model.predict([X_test_img,X_test_auxiliary1,X_test_auxiliary2,X_test_auxiliary3])
y_pred = y_pred1.reshape(-1)
MAE=mae(y_test,y_pred)
RMSE=mse(y_test,y_pred,squared=False)
MAPE=mape(y_test,y_pred)
R2=rSquared(y_test,y_pred)

print("Mean Absolute Error:", MAE)
print("Root Mean Squared Error:", RMSE)
print("Mean Absolute Precentage Error",MAPE)
print("Coefficient of determination ",R2)

In [ ]:
import datetime
x=datetime.datetime.now()
model_name=("{}_{}_{}_{}.hdf5".format(x.month,x.day,x.hour,x.minute))

model.save('/kaggle/working/'+model_name)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting y_test values
plt.scatter(y_pred, y_test, c='blue', label='True Values')

max_value = max(max(y_test), max(y_pred))
min_value = min(min(y_test), min(y_pred))
plt.plot([min_value, max_value], [min_value, max_value], color='red',linestyle='dashed',label='Reference line')


plt.xlabel('Predicted PM2.5 values (ug/m3)', fontsize=15)
plt.ylabel('True PM2.5 values (ug/m3)', fontsize=15)
#plt.legend()
#plt.title("Scatter plot of Predicted Vs True PM2.5 Concentrations")
plt.show()
